# Agent Demo with Simplified Connection Management

This notebook demonstrates an agent using multiple tool types with simplified connection management:
1. **Local Calculator**: Mathematical expressions via MCP
2. **Brave Search**: Web search via MCP (if API key available)
3. **Think Tool**: Internal reasoning tool

In [1]:
import os
import sys
import nest_asyncio
nest_asyncio.apply()

parent_dir = os.path.dirname(os.getcwd())
sys.path.insert(0, parent_dir)
from agents.agent import Agent, ModelConfig
from agents.tools.think import ThinkTool
from agents.tools.web_search import WebSearchServerTool
from agents.tools.code_execution import CodeExecutionServerTool

# 1. Configure tools and MCP servers

In [2]:
# Standard Python tool
think_tool = ThinkTool()

# Python MCP server
calculator_server_path = os.path.abspath(os.path.join(os.getcwd(), "tools/calculator_mcp.py"))
calculator_server = {
    "type": "stdio",
    "command": "python",
    "args": [calculator_server_path]
}
print(f"Calculator server configured: {'Yes' if calculator_server else 'No'}")


# Brave MCP server written in TypeScript

brave_api_key = os.environ.get("BRAVE_API_KEY_BASE_DATA", "")
print(f"Brave API key available: {'Yes' if brave_api_key else 'No'}")
brave_search_server = {
    "type": "stdio",
    "command": "npx",
    "args": ["-y", "@modelcontextprotocol/server-brave-search"],
    "env": {
        "BRAVE_API_KEY": brave_api_key, 
        "PATH": f"{os.path.dirname('npx')}:" + os.environ.get("PATH", "")
    }
}
print(f"Brave search server configured: {'Yes' if brave_search_server else 'No'}")

Calculator server configured: Yes
Brave API key available: Yes
Brave search server configured: Yes


# 2. Initialize agent with combined tools

In [3]:
# Create agent config 
system_prompt = """
You are a helpful assistant with access to:
1. Web search (brave_web_search, brave_local_search)
2. Mathematical calculator (calculate) 
3. A tool to think and reason (think)

Always use the most appropriate tool for each task.
"""


# Initialize agent with standard tools and MCP servers
agent = Agent(
    name="Multi-Tool Agent",
    system=system_prompt,
    tools=[think_tool], 
    mcp_servers=[brave_search_server, calculator_server], 
    config=ModelConfig(
        model="claude-3-7-sonnet-20250219", 
        max_tokens=4096,
        temperature=1.0
    ),
    verbose=True
)


[Multi-Tool Agent] Agent initialized


# 3. Run queries

In [4]:
# Example query
agent.run("What's the square root of the OKC population in 2022")

Loaded 3 MCP tools from 2 servers.

[Multi-Tool Agent] Received: What's the square root of the OKC population in 2022

[Multi-Tool Agent] Output: I'll help you find the square root of Oklahoma City's population in 2022. First, I need to look up the population data, then I can calculate the square root.

[Multi-Tool Agent] Tool call: brave_web_search(query=Oklahoma City population 2022 census data)

[Multi-Tool Agent] Tool result: Title: Oklahoma Cities by Population (2025)
Description: The most populous cities in Oklahoma are <strong>Oklahoma City with 702,767</strong>, Tulsa at 411,894, Norman with 130,046, Broken Arrow at 119,194, and Edmond with 98,103. You can get the most current Census population data for all 834 Oklahoma cities, sorted from the most populous cities in Oklahoma to the ...
URL: https://www.oklahoma-demographics.com/cities_by_population

Title: U.S. Census Bureau QuickFacts: Oklahoma City city, Oklahoma
Description: QuickFacts provides statistics for all states and


[Multi-Tool Agent] Output: I still don't see a specific figure for exactly 2022, but I can gather enough information to make a reasonable estimate.

[Multi-Tool Agent] Tool call: think(thought=From the search results, I can see:
1. The 2020 census population was 681,054
2. Oklahoma City grew by about 1% between 2022 and 2023, adding 7,589 people
3. This implies the 2022 population was approximately 7,589/0.01 = 758,900 people
4. But this seems too high compared to other figures like 702,767 and 688,700 mentioned

Let me reconsider. If Oklahoma City added 7,589 people between 2022-2023 and that represented a 1% growth, then the 2022 population would be:
7,589 ÷ 0.01 = 758,900

However, this contradicts other data points. Perhaps the clearest estimate is from the second search result which states: "The metro area population of Oklahoma City in 2022 was 1,008,000". But this refers to the metro area (larger than just the city proper).

Another data point from the first search states "Okla

Message(id='msg_016wLY8JCd5baUgaWEHcrage', content=[TextBlock(text="The square root of Oklahoma City's population in 2022 (estimated at 696,272 people) is approximately 834.43.", type='text')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=5, output_tokens=34, cache_creation_input_tokens=150, cache_read_input_tokens=3881, service_tier='standard'))

In [5]:
await agent.run_async("How many bananas will fit in an Toyota GR86?")

Loaded 3 MCP tools from 2 servers.

[Multi-Tool Agent] Received: How many bananas will fit in an Toyota GR86?

[Multi-Tool Agent] Output: That's an interesting question! To answer how many bananas will fit in a Toyota GR86, I'll need to find information about the car's cargo capacity and then estimate how many bananas could fit in that space.

[Multi-Tool Agent] Tool call: brave_web_search(query=Toyota GR86 cargo capacity trunk volume)

[Multi-Tool Agent] Tool result: Title: Cargo space. What can it fit? : r/GR86
Description: 11 votes, 36 comments. 42K subscribers in the <strong>GR86</strong> community. For anyone interested in the <strong>Toyota</strong> <strong>GR86</strong> &amp; Subaru BRZ (&#x27;22+).
URL: https://www.reddit.com/r/GR86/comments/16xh6ii/cargo_space_what_can_it_fit/

Title: 2024 Toyota GR86 Review, Pricing, and Specs
Description: The 2024 <strong>Toyota</strong> <strong>GR86</strong> turns every day into an autocross day thanks to its playful chassis and easy-to-sli


[Multi-Tool Agent] Output: Now I have the information I need to calculate how many bananas will fit in a Toyota GR86. Let me think through this:

[Multi-Tool Agent] Tool call: think(thought=From the search results, I found:

1. Toyota GR86 trunk capacity:
   - 6.3 cubic feet of trunk space (according to multiple sources)
   - This equals approximately 178,385 cubic centimeters (6.3 ft³ × 28,316.85 cm³/ft³)

2. Average banana volume:
   - A fully ripe Cavendish banana has a volume of approximately 156.10 cubic centimeters

3. Packing efficiency:
   - When packing irregular objects like bananas, we need to account for the space between them
   - A Reddit comment mentioned packing efficiency of around 65% for bananas
   - This means only about 65% of the total volume can be filled with bananas due to gaps

So to calculate how many bananas will fit in the Toyota GR86 trunk:
- Available volume with packing efficiency: 178,385 cm³ × 0.65 = 115,950 cm³
- Number of bananas: 115,950 cm³ ÷ 156.

Message(id='msg_01NC46TLNZSG97ZH5v7b7YcN', content=[TextBlock(text="# How Many Bananas Will Fit in a Toyota GR86\n\nBased on my research, I can provide you with an estimate of how many bananas would fit in a Toyota GR86.\n\n## The Data:\n\n- **Toyota GR86 trunk capacity**: 6.3 cubic feet (approximately 178,385 cubic centimeters)\n- **Toyota GR86 passenger volume**: 78 cubic feet\n- **Average banana volume**: 156.10 cubic centimeters per banana\n- **Packing efficiency**: Approximately 65% (accounting for inevitable gaps between irregularly shaped bananas)\n\n## The Calculation:\n\nIf we're talking about filling just the trunk:\n- Available volume with packing efficiency: 178,385 cm³ × 0.65 = 115,950 cm³\n- Number of bananas in trunk: 115,950 cm³ ÷ 156.10 cm³ = approximately **743 bananas**\n\nIf we're considering the entire car (including passenger compartment):\n- Total car volume: 84.3 cubic feet (2,387,110 cm³)\n- Available volume with packing efficiency: 2,387,110 cm³ × 0.65 = 1,551

# 4. Example with Anthropic Server Tools

This example demonstrates using Anthropic's native server tools for web search and code execution.

In [9]:
# Create Anthropic server tools
web_search_tool = WebSearchServerTool(
    name="web_search",
    max_uses=5,  # Limit to 5 searches per request
    blocked_domains=["example.com"]  # Example of blocking specific domains
)

code_execution_tool = CodeExecutionServerTool()

# Initialize agent with server tools
server_agent = Agent(
    name="Server Tools Agent",
    system="""
You are a helpful assistant with access to:
1. Web search for finding current information
2. Code execution for running Python code
3. Think tool for complex reasoning

Use these tools effectively to answer questions that require current data or calculations.
""",
    tools=[think_tool, web_search_tool, code_execution_tool],
    config=ModelConfig(
        model="claude-sonnet-4-20250514",
        max_tokens=4096,
        temperature=0.7
    ),
    verbose=True
)


[Server Tools Agent] Agent initialized


In [10]:
# Example 1: Use web search to find current information and code execution for analysis
server_agent.run("""
Search for the current population of Tokyo, Japan. 
Then write and execute Python code to calculate how many people that would be per square kilometer, 
given that Tokyo's area is approximately 2,194 square kilometers.
""")


[Server Tools Agent] Received: 
Search for the current population of Tokyo, Japan. 
Then write and execute Python code to calculate how many people that would be per square kilometer, 
given that Tokyo's area is approximately 2,194 square kilometers.


[Server Tools Agent] Output: I'll help you find Tokyo's current population and then calculate the population density. Let me start by searching for the current population data.

[Server Tools Agent] Output: Now let me search for more specific information about Tokyo's current population, as the results show different figures for different definitions of "Tokyo" (city proper vs metropolitan area).

[Server Tools Agent] Tool call: think(thought=Now I have a clearer picture of Tokyo's population. From the search results, I can see there are different definitions of "Tokyo":

1. Tokyo Metropolis (Tokyo-to) - the prefecture/administrative area: Around 14 million people as of 2024
2. The 23 Special Wards (central Tokyo): Around 9.2-9.7 millio

Message(id='msg_018nAgEXuVMSJCTSrSjcZtrj', content=[TextBlock(text="Now I'll write and execute Python code to calculate Tokyo's population density based on the information I found.", type='text'), TextBlock(text=None, type='server_tool_use', id='srvtoolu_01JAxi9fcajoScWQWEG1bJ2S', name='code_execution', input={'code': '# Tokyo population and area data\n# Based on search results, Tokyo Metropolis (Tokyo-to) has approximately 14 million people\n# and covers 2,194 square kilometers\n\ntokyo_population = 14_000_000  # Current population of Tokyo Metropolis (2024)\ntokyo_area_sq_km = 2194  # Area in square kilometers\n\n# Calculate population density\npopulation_density = tokyo_population / tokyo_area_sq_km\n\nprint("Tokyo Population Density Calculation")\nprint("=" * 40)\nprint(f"Tokyo Metropolis Population: {tokyo_population:,} people")\nprint(f"Tokyo Metropolis Area: {tokyo_area_sq_km:,} square kilometers")\nprint(f"Population Density: {population_density:,.2f} people per square kilomete